RNN

In [1]:
import numpy as np
import pandas as pd

from gensim.models import Word2Vec
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder 

In [2]:
# create dataset
data = {
    "text": [
        "Otelin konumu mükemmeldi, her yere yürüyerek ulaştık.",
        "Çalışanlar çok soğuktu, kendimizi rahatsız hissettik.",
        "Odalar çok temizdi, her gün düzenli temizlik yapıldı.",
        "Kahvaltı çeşitleri çok azdı ve lezzetsizdi.",
        "Manzara harikaydı, sabahları güneşin doğuşunu izlemek muhteşemdi.",
        "Oda çok küçüktü, bavullarımızı bile zor açtık.",
        "Spa ve sauna hizmetleri çok iyiydi, çok rahatladık.",
        "Resepsiyondaki görevliler ilgisizdi, defalarca beklemek zorunda kaldık.",
        "Yataklar çok rahattı, her gece deliksiz uyuduk.",
        "Duş başlığı kırık ve su basıncı çok düşüktü.",
        "Otel restoranı muazzamdı, yemeklerin tadı inanılmazdı.",
        "Oda servisi çok yavaştı, bir sandviç için 1 saat bekledik.",
        "Otelin bahçesi ve peyzajı harikaydı, adeta bir cennet.",
        "Klimamız çalışmıyordu, sıcak yüzünden uyuyamadık.",
        "Personel her konuda çok yardımcıydı, teşekkür ederiz.",
        "Otel çok eskiydi, yenileme şart.",
        "Havuz suyu temiz ve ılık, çocuklar da çok eğlendi.",
        "Odada rutubet kokusu vardı, rahatsız ediciydi.",
        "Giriş işlemleri çok hızlı yapıldı, beklemedik.",
        "Minibar tamamen boştu, içecek bile yoktu.",
        "Otel oldukça sessizdi, huzurlu bir tatil geçirdik.",
        "Wi-Fi bağlantısı kopup duruyordu, çalışmak imkansızdı.",
        "Yemeklerde hem çeşit boldu hem de hepsi çok lezzetliydi.",
        "Kahvaltı salonu çok kalabalıktı, yer bulmakta zorlandık.",
        "Çalışanlar çok güler yüzlü ve saygılıydı.",
        "Odamızın penceresinden çöplük görünüyordu, çok kötüydü.",
        "Denize sıfır konumdaydı, harika bir tatil geçirdik.",
        "Check-in sırasında uzun süre beklemek zorunda kaldık.",
        "Odaların dekorasyonu modern ve şıktı.",
        "Restoran fiyatları gereksiz pahalıydı.",
        "Çocuklar için oyun alanı çok geniş ve güvenliydi.",
        "Asansör çok eskiydi ve korkutucuydu.",
        "Otelin sunduğu turlar çok eğlenceliydi.",
        "Odaya girdiğimizde yatak örtüsü kirliydi.",
        "Müşteri hizmetleri sorunlarımızı anında çözdü.",
        "Havuz çevresi çok kaygandı, tehlikeli olabilirdi.",
        "Otelin lobisi çok şıktı ve ferah bir hava veriyordu.",
        "Akşam yemeklerinde sürekli aynı yemekler çıkıyordu.",
        "Otel personeli çok ilgiliydi, her ihtiyacımızı giderdiler.",
        "Duşta sıcak su akmıyordu, zor anlar yaşadık.",
        "Manzara fotoğraflardaki gibiydi, gerçek bir görsel şölen.",
        "Yatak çarşaflarında lekeler vardı, değiştirmek zorunda kaldık.",
        "Bahçe katı odalar böcek doluydu.",
        "Otelin aktiviteleri çok çeşitli ve eğlenceliydi.",
        "Oda kapımız tam kapanmıyordu, güvenlik sorunu vardı.",
        "Deniz manzaralı odamız rüya gibiydi.",
        "Oda servisi çok geç geldi, şikayet etmek zorunda kaldık.",
        "Bar bölümündeki kokteyller çok başarılıydı.",
        "Resepsiyonist çok kaba davrandı, hiç hoş değildi.",
        "Konfor açısından otel mükemmeldi.",
        "Banyo havluları yırtıktı, değiştirilmesi gerekiyor.",
        "Otel plajı kumluydu ve çok temizdi.",
        "Sıcak su sık sık kesiliyordu, çok rahatsız ediciydi.",
        "Tatil boyunca hiç sorun yaşamadık, her şey çok güzeldi.",
        "Asansör sürekli arızalanıyordu.",
        "Her sabah oda servisi kahvaltımızı zamanında getirdi.",
        "Koridorlar karanlık ve ürkütücüydü.",
        "Tüm personel çok profesyoneldi.",
        "Odada kötü bir koku vardı, havalandırmak zorunda kaldık.",
        "Kumsalda ücretsiz şezlong olması büyük avantajdı.",
        "Odaya ilk girdiğimizde çöp kovası doluydu.",
        "Fitness salonu çok bakımsızdı, aletler çalışmıyordu.",
        "Otel konsepti çocuklu aileler için çok uygundu.",
        "Yemek saatlerinde restoran inanılmaz kalabalık oluyordu.",
        "Her akşam farklı aktiviteler vardı, çok eğlendik.",
        "Odalar fotoğraflardakinden çok farklıydı, hayal kırıklığı yaşadık.",
        "Personel maske ve hijyen kurallarına dikkat ediyordu.",
        "Havuz suyu bulanıktı, temiz değildi.",
        "Otelin sunduğu kahvaltı açık büfe ve çok lezzetliydi.",
        "Gece yarısı odada su kesildi, duş yapamadık.",
        "Restoran bölümü manzaralı ve çok şıktı.",
        "Klima kumandası çalışmıyordu, odamız sıcaktı.",
        "Güler yüzlü personel sayesinde kendimizi çok özel hissettik.",
        "Odada prizler çalışmıyordu, şarj sorunu yaşadık.",
        "Havuz etrafı hijyenikti, sürekli temizlik yapılıyordu.",
        "Otele girişte valizlerimiz ortada bekletildi.",
        "Yataklar yeni ve çok rahattı.",
        "Otel resepsiyonu çok karışıktı, kimse ne yaptığını bilmiyordu.",
        "Akşam yemeği için canlı müzik vardı, çok keyifliydi.",
        "Yatak odasında rutubet vardı, duvarlar nemliydi.",
        "Çocuklar için animasyon gösterileri vardı, çok başarılıydı.",
        "Restoran personeli çok ilgisizdi.",
        "Otelin mimarisi modern ve şıktı.",
        "Banyoda sıcak su geç geliyordu.",
        "Deniz suyunun temizliği harikaydı.",
        "Odalarda sinek vardı, rahat edemedik.",
        "Otelin sunduğu shuttle hizmeti çok konforluydu.",
        "Çarşaflar temiz ve mis gibi kokuyordu.",
        "Oda kartı sürekli çalışmıyordu, değiştirmek zorunda kaldık.",
        "Plajda yer bulmak çok zordu, yetersiz şezlong vardı.",
        "Otel çalışanları sorunlara hızlı çözümler sundu.",
        "Çocuk havuzu çok küçük ve yetersizdi.",
        "Check-out işlemleri çok hızlıydı, beklemeden ayrıldık.",
        "Çalışanlar İngilizce bilmiyordu, iletişim zor oldu.",
        "Otel çevresi çok sessiz ve huzurluydu.",
        "Girişte arabamız için vale hizmeti sunulmadı.",
        "Otelin sabah kahvaltısı çok taze ve zengindi.",
        "Banyoda küf vardı, temizlik yetersizdi.",
        "Otelin ışıkları çok iyidi.",
        "Banyoda duşjeli yoktu, yıkanamadık.",
    ],
    "label": [
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif",
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif",
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif",  
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif",
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif", 
        "pozitif", "negatif", "pozitif", "negatif",
    ]
}

In [3]:
print(len(data["text"]))   # 100 olmalı
print(len(data["label"]))  # 100 olmalı

100
100


In [4]:
df = pd.DataFrame(data)

In [5]:
df.head()

,text,label
0,"Otelin konumu mükemmeldi, her yere yürüyerek u...",pozitif
1,"Çalışanlar çok soğuktu, kendimizi rahatsız his...",negatif
2,"Odalar çok temizdi, her gün düzenli temizlik y...",pozitif
3,Kahvaltı çeşitleri çok azdı ve lezzetsizdi.,negatif
4,"Manzara harikaydı, sabahları güneşin doğuşunu ...",pozitif


In [6]:
# metin verisi tokenize
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["text"])
sequences = tokenizer.texts_to_sequences(df["text"])
word_index = tokenizer.word_index
print("Vocab size: ", len(word_index))

Vocab size:  387


In [7]:
# padding process
maxlen = max(len(seq) for seq in sequences)
x = pad_sequences(sequences, maxlen = maxlen)
print("X shape: ", x.shape)

X shape:  (100, 10)


In [8]:
# label encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df["label"])
print("Y shape: ", y.shape)

Y shape:  (100,)


In [9]:
# train-test split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

word embeding: word2vec, embedding matrisi olusturma

In [10]:
sentences = [text.split() for text in df["text"]]
print(sentences)

[['Otelin', 'konumu', 'mükemmeldi,', 'her', 'yere', 'yürüyerek', 'ulaştık.'], ['Çalışanlar', 'çok', 'soğuktu,', 'kendimizi', 'rahatsız', 'hissettik.'], ['Odalar', 'çok', 'temizdi,', 'her', 'gün', 'düzenli', 'temizlik', 'yapıldı.'], ['Kahvaltı', 'çeşitleri', 'çok', 'azdı', 've', 'lezzetsizdi.'], ['Manzara', 'harikaydı,', 'sabahları', 'güneşin', 'doğuşunu', 'izlemek', 'muhteşemdi.'], ['Oda', 'çok', 'küçüktü,', 'bavullarımızı', 'bile', 'zor', 'açtık.'], ['Spa', 've', 'sauna', 'hizmetleri', 'çok', 'iyiydi,', 'çok', 'rahatladık.'], ['Resepsiyondaki', 'görevliler', 'ilgisizdi,', 'defalarca', 'beklemek', 'zorunda', 'kaldık.'], ['Yataklar', 'çok', 'rahattı,', 'her', 'gece', 'deliksiz', 'uyuduk.'], ['Duş', 'başlığı', 'kırık', 've', 'su', 'basıncı', 'çok', 'düşüktü.'], ['Otel', 'restoranı', 'muazzamdı,', 'yemeklerin', 'tadı', 'inanılmazdı.'], ['Oda', 'servisi', 'çok', 'yavaştı,', 'bir', 'sandviç', 'için', '1', 'saat', 'bekledik.'], ['Otelin', 'bahçesi', 've', 'peyzajı', 'harikaydı,', 'adeta', 'b

In [11]:
word2vec_model = Word2Vec(sentences, vector_size = 64, window = 5, min_count = 1)

In [12]:
# içi 0 larla dolu matrix yapiyoruz
embedding_dim =64
embedding_matrix = np.zeros(( len(word_index) + 1, embedding_dim))
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
# içini dolduralim
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]

embedding_matrix

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-6.08499860e-04,  5.69149488e-05,  8.05266108e-03, ...,
         4.38622752e-04, -6.37325738e-03, -1.19741773e-02],
       [-2.08232505e-03,  3.58247152e-03, -1.31548976e-03, ...,
         4.46957257e-03, -7.57963257e-03,  6.93820976e-03],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.68378295e-03, -1.14280023e-02, -1.49940746e-02, ...,
        -3.77684855e-03, -1.21133951e-02,  1.07869916e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

Build RNN Model

In [14]:
model = Sequential()
model.add(Embedding(input_dim = len(word_index)+1, output_dim = embedding_dim, weights = [embedding_matrix], input_length = maxlen, trainable = False)) 
# ogrenme yok diye trainableyi false yaptik. eğitim RNN içinde olcak

model.add(SimpleRNN(32, return_sequences= False))
model.add(Dense(1, activation= "sigmoid"))

model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

Train RNN model

In [15]:
#early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True) # eklemeye gerek yok gibi amacmız gözlemlemek zaten

In [16]:
model.fit(x_train, y_train, epochs = 20, batch_size = 8, validation_data = (x_test, y_test),)  #callbacks = [early_stopping]

Epoch 1/20
9/9 [==============================] - 2s 55ms/step - loss: 0.6929 - accuracy: 0.4429 - val_loss: 0.6975 - val_accuracy: 0.3667
Epoch 2/20
9/9 [==============================] - 0s 8ms/step - loss: 0.6879 - accuracy: 0.6143 - val_loss: 0.6986 - val_accuracy: 0.4333
Epoch 3/20
9/9 [==============================] - 0s 8ms/step - loss: 0.6833 - accuracy: 0.5857 - val_loss: 0.7006 - val_accuracy: 0.4333
Epoch 4/20
9/9 [==============================] - 0s 8ms/step - loss: 0.6794 - accuracy: 0.5714 - val_loss: 0.7081 - val_accuracy: 0.4000
Epoch 5/20
9/9 [==============================] - 0s 8ms/step - loss: 0.6727 - accuracy: 0.5857 - val_loss: 0.7063 - val_accuracy: 0.4667
Epoch 6/20
9/9 [==============================] - 0s 8ms/step - loss: 0.6654 - accuracy: 0.6000 - val_loss: 0.7117 - val_accuracy: 0.4333
Epoch 7/20
9/9 [==============================] - 0s 8ms/step - loss: 0.6580 - accuracy: 0.5857 - val_loss: 0.7162 - val_accuracy: 0.4333
Epoch 8/20
9/9 [=================

Evaluate RNN model

In [17]:
print(" ")
loss, accuracy = model.evaluate(x_test, y_test)
print("Test loss: ", loss)
print("Test accuracy: ", accuracy)

 
1/1 [==============================] - 0s 25ms/step - loss: 0.9661 - accuracy: 0.5000
Test loss:  0.9660879969596863
Test accuracy:  0.5


loss azalmiş, accuracy artmış bu ikssi doğru\
ama vall_loss azaltmasi gerekn yerde artmış, vall accuracy de sonaa azalmiş, bunu sebebi yetersiz veri setimizin olması

Cümle Sınıflandırma Çalışması

In [18]:
def classify_sentence(sentence):
    seq = tokenizer.texts_to_sequences([sentence])
    padded_seq = pad_sequences(seq, maxlen = maxlen)

    prediction = model.predict(padded_seq)
    predicted_class = (prediction > 0.5).astype(int)
    label = "pozitif" if predicted_class[0][0] == 1 else "negatif"
    return label

In [19]:
sentence = "Odada klima çok gürültülüydü, rahatsız olduk"
result = classify_sentence(sentence)
print("Tahmin: ", result)

1/1 [==============================] - 0s 266ms/step
Tahmin:  negatif


In [20]:
sentence = "Otel çok temiz ve rahattı, çok memnun kaldık."
result = classify_sentence(sentence)
print("Tahmin: ", result)

1/1 [==============================] - 0s 21ms/step
Tahmin:  pozitif


Ödev:
Verisetini 1k yap bulursan\
test_size (0.1, 0.2, 0.3) gibi değiş\
SimpleRNN() deki cell sayisini değiştir(nöron sayimiz bu, ne kadar çok o kadar iyi öğrenir yada ezberler) \
epoch değiştir\
batch_siz değiştir\